<a href="https://colab.research.google.com/github/MTBXC/MTBXC/blob/main/Automatic_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install python-dotenv
import kagglehub
import os
from dotenv import load_dotenv

# Zdefiniuj ścieżkę do pliku .env na Twoim Google Drive
dotenv_path = '/content/drive/MyDrive/API_KEY/kaggle.env'

# Sprawdź, czy plik istnieje przed próbą jego wczytania
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)

    # Pobierz zmienne środowiskowe
    kaggle_login = os.getenv('KAGGLE_LOGIN')
    kaggle_key = os.getenv('KAGGLE_KEY')

    # # Możesz teraz użyć kaggle_login i kaggle_key
    # print(f"KAGGLE_LOGIN: {kaggle_login}")
    # print(f"KAGGLE_KEY: {kaggle_key}")
else:
    print(f"Plik .env nie został znaleziony w {dotenv_path}")
os.environ['KAGGLE_USERNAME'] = kaggle_login
os.environ['KAGGLE_KEY'] = kaggle_key

kagglehub.login()



Mounted at /content/drive


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

print('Importing data sources...')

titanic_path = kagglehub.competition_download('titanic')

print('Data source import complete.')


Importing data sources...


100%|██████████| 34.1k/34.1k [00:00<00:00, 16.9MB/s]

Extracting files...
Data source import complete.


In [3]:
# List files in the downloaded directory
print(f"Files in the downloaded directory ({titanic_path}):")
for dirname, _, filenames in os.walk(titanic_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Files in the downloaded directory (/root/.cache/kagglehub/competitions/titanic):
/root/.cache/kagglehub/competitions/titanic/gender_submission.csv
/root/.cache/kagglehub/competitions/titanic/train.csv
/root/.cache/kagglehub/competitions/titanic/test.csv


In [5]:
import pandas as pd
import numpy as np

train_data = pd.read_csv(os.path.join(titanic_path, "train.csv"))
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
mean_age = train_data['Age'].mean()
train_data['Age'] = train_data['Age'].fillna(mean_age)
print(train_data.isna().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [19]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np

# Prepare data (same as before)
y = train_data["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

# Initialize KFold
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)

# Store predictions from each fold
oof_predictions = np.zeros(len(train_data))
test_predictions = np.zeros(len(test_data))
models = []
oof_scores = []

# Loop through each fold
for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
    print(f"Fold {fold+1}/{n_splits}")

    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Initialize and train LightGBM model
    lgbm = lgb.LGBMClassifier(random_state=1)
    lgbm.fit(X_train, y_train,
             eval_set=[(X_val, y_val)]) # Set to True for more detailed output during training

    # Make predictions
    val_preds = lgbm.predict(X_val)
    test_preds = lgbm.predict(X_test)

    # Store predictions and score
    oof_predictions[val_index] = val_preds
    test_predictions += test_preds / n_splits # Average predictions across folds
    models.append(lgbm)

    score = accuracy_score(y_val, val_preds)
    oof_scores.append(score)
    print(f"Fold {fold+1} OOF Accuracy: {score}")

# Evaluate overall OOF predictions
overall_oof_accuracy = accuracy_score(y, oof_predictions)
print(f"\nOverall OOF Accuracy: {overall_oof_accuracy}")
print(f"Average Fold OOF Accuracy: {np.mean(oof_scores)}")

# Create submission file using averaged test predictions
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': test_predictions.round().astype(int)})
output.to_csv('submission_lgbm_kf.csv', index=False)

print("\nYour LightGBM with KFold submission was successfully saved!")

Fold 1/5
[LightGBM] [Info] Number of positive: 269, number of negative: 443
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000381 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 21
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.377809 -> initscore=-0.498858
[LightGBM] [Info] Start training from score -0.498858
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [20]:
!kaggle competitions submit -c titanic -f submission_lgbm_kf.csv -m "Submission using Google Colab with LGBM"

100% 2.77k/2.77k [00:00<00:00, 11.1kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster